http://nl.ijs.si/jaslo/index-en.html#jpwac

https://www.clarin.si/ske/#concordance?corpname=jpwac_l4

The corpus contains over 300 million words, with annotations of words and sentences describing their difficulty levels. Words are assigned levels of difficulty according to the Japanese Language Proficiency Test Content Specifications (2004). The difficulty level of the sentences is computed using various heuristics, based on the (difficulty level of) words, sentence length, etc. The corpus was collected from the Web using WaCkY tools, part-of-speech tagged and lemmatised with Chasen. The Japanese Chasen tags have also been converted to English language based tags.  The corpora are made available in vertical format. Structural attributes are text and s (sentence). 
- Each text gives its @url and @domain. 
- Sentences have the @level attribute, which describes their difficulty level. 
The positional attributes are:  
1. token, as it appears in the text 
2. lemma of the word 
3. Chasen tag, translated to English 
4. original Chasen tag in Japanese 
5. difficulty level of the word.  

The complete corpus is also split into sub-corpora of sentences with the same difficulty level.

- 0 - very difficult (N1)
- 1 - difficult (N2)
- 2 - intermediate (N3)
- 3 - easy (N4)
- 4 - very easy (N5) -> not really, it was from when the jlpt had only 4 levels. The paper tells about the criteria.

we could probably substitute them to make it easier. Problem is that punctuation and particles are 0... we should check what other things are given 0 in the dataset. -> words not appearing in the list of the JLPT 2004



gunzip -c  "jpWaC-L0.vert.gz" | wc -l
51691509 -> around 5 million sentences IG [not really]

read on its own = takes 5 minutes... maybe it's better to read in chunks with pandas iterparse argument in read_xml

1) count the number of sentences and maybe sample a little less because data is very imbalanced
actually its okay... but those sentences are so long maybe?

- size of L0: 46461
- size of L1: 31233
- size of L2: 37365
- size of L3: 23314
- size of L4: 14316

doing level counts.. (weird af)
-1       9211681
 0       2627335
 2        372421
 1        137312
 3        103298
 4         36563

In [13]:
import sys
import os 




'ipykernel_launcher.py'

In [38]:
import logging

# Create a logger
logger = logging.getLogger(os.path.basename(sys.argv[0]))

# Set the logging level (choose one of: DEBUG, INFO, WARNING, ERROR, CRITICAL)
logger.setLevel(logging.DEBUG)

# Create a file handler to save logs to a file
file_handler = logging.FileHandler('logs.log')

# Create a formatter to specify the log message format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Example function that logs messages
def my_function():
    logger.debug('This is a debug message.')
    logger.info('This is an info message.')
    logger.warning('This is a warning message.')
    logger.error('This is an error message.')
    logger.critical('This is a critical message.')

# Call the function to generate logs
my_function()


In [1]:
import gzip
import pandas as pd
import re


df = None

with gzip.open('../../data/jpWaC/jpWaC-L4.vert.gz','rt') as file:
    df = pd.read_xml(file)

In [44]:
def process_data_jpwac(file_name):
    df = None

    file_name = os.path.realpath(file_name)
    dir_name = os.path.dirname(file_name)

    logger.debug(f'opening {file_name}')
    with gzip.open(file_name, 'rt') as file:
        # read whole file, may take a while
        df = pd.read_xml(file)
    
    df = df.drop(columns=['domain', 'gap'])

    # get the level from filename
    base_name = os.path.basename(file_name)
    match = re.search(r'(\d+)', base_name)
    if match:
        annotated_level = match.group(1)
    else:
        annotated_level = None
    logger.debug(annotated_level)
    df['level_old_jlpt'] = annotated_level

    df_processed = df
    df_processed['sentence'] = df['s'].map(extract_sentence)
    df_processed = df_processed.drop(columns='s')

    # saving to file
    df_base_name = base_name.split('.')[0] + '.csv'
    logger.debug(df_base_name)
    df_file_name = os.path.join(dir_name, df_base_name)
    logger.info(f'saving to {df_file_name}')
    
    df_processed.to_csv(df_file_name)

    logger.debug(f'columns are {df_processed.columns.values}')
    
    return df_processed

In [45]:
process_data_jpwac('../../data/jpWaC/jpWaC-L4.vert.gz')

In [2]:
df = df.drop(columns=['domain', 'gap'])

In [3]:
df['level_old_jlpt'] = 4
df

,url,s,level_old_jlpt
0,http://www.01-tec.com/document/code_design/vol...,\nよろしく\tよろしく\tAdv.g\t副詞-一般\t4\nお願い\tお願い\tN.Vs\...,4
1,http://www.0karakasegu.com/,\nいい\tいい\tAi.free\t形容詞-自立\t4\n勉強\t勉強\tN.Vs\t名詞...,4
2,http://000studio.com/bunka/main/agora/agora02....,\nそれ\tそれ\tN.Pron.g\t名詞-代名詞-一般\t4\nを\tを\tP.c.g\...,4
3,http://www.01-tec.com/book_shop/,\n１\t１\tN.Num\t名詞-数\t0\n冊\t冊\tN.Suff.msr\t名詞-接...,4
4,http://www.1101.com/saeki/archive/2000-03-22.html,\nなん\tなる\tV.free\t動詞-自立\t4\nな\tだ\tAux\t助動詞\t4\...,4
...,...,...,...
14311,http://zjq.sakura.ne.jp/rawi/the-non-repetitiv...,\n新しい\t新しい\tAi.free\t形容詞-自立\t4\n1\t1\tN.Num\t名...,4
14312,http://www.zushi-kaisei.ac.jp/activity/study/s...,\nまた\tまた\tConj\t接続詞\t4\n、\t、\tSym.c\t記号-読点\t0\...,4
14313,http://zerowaste.jp/cyberaction/nowar/kzm2004/...,\n私\t私\tN.Pron.g\t名詞-代名詞-一般\t4\nも\tも\tP.bind\t...,4
14314,http://zerowaste.jp/cyberaction/nowar/kzm2004/...,\nよく\tよく\tAdv.g\t副詞-一般\t4\nわかり\tわかる\tV.free\t動...,4


In [4]:
test_sent = df.loc[10]['s']
test_sent

'\nっ\tく\tV.bnd\t動詞-非自立\t4\n言っ\t言う\tV.free\t動詞-自立\t4\nて\tて\tP.Conj\t助詞-接続助詞\t4\nい\tいる\tV.bnd\t動詞-非自立\t4\nた\tた\tAux\t助動詞\t4\n。\t。\tSym.p\t記号-句点\t0\n'

In [5]:
import re

# def extract_japanese_sentence1(input_str):
#     # Define a regular expression pattern to match Japanese words
#     pattern = r'([^\t\n]+)\t[^\t\n]+\t[^\t\n]+\t[^\t\n]+\t[0-9]+\n'

#     # Use re.findall to extract all matches
#     japanese_words = re.findall(pattern, input_str)

#     # Join the extracted words to form the Japanese sentence
#     japanese_sentence = ''.join(japanese_words)

#     # Remove any leading/trailing whitespaces and line breaks
#     japanese_sentence = japanese_sentence.strip()

#     return japanese_sentence

# # Example usage:

# japanese_sentence = extract_japanese_sentence(test_sent)
# print(japanese_sentence)


def extract_sentence(input_str):
    """Processes the column formulation into the original sentence."""
    char_list = input_str.split('\n')
    sentence_chars = [char.split('\t')[0] for char in char_list]
    return "".join(sentence_chars)


In [6]:
for i in range(10000):
    test_sent = df.loc[i]['s']
    jp = extract_sentence(test_sent)

In [7]:
df

,url,s,level_old_jlpt
0,http://www.01-tec.com/document/code_design/vol...,\nよろしく\tよろしく\tAdv.g\t副詞-一般\t4\nお願い\tお願い\tN.Vs\...,4
1,http://www.0karakasegu.com/,\nいい\tいい\tAi.free\t形容詞-自立\t4\n勉強\t勉強\tN.Vs\t名詞...,4
2,http://000studio.com/bunka/main/agora/agora02....,\nそれ\tそれ\tN.Pron.g\t名詞-代名詞-一般\t4\nを\tを\tP.c.g\...,4
3,http://www.01-tec.com/book_shop/,\n１\t１\tN.Num\t名詞-数\t0\n冊\t冊\tN.Suff.msr\t名詞-接...,4
4,http://www.1101.com/saeki/archive/2000-03-22.html,\nなん\tなる\tV.free\t動詞-自立\t4\nな\tだ\tAux\t助動詞\t4\...,4
...,...,...,...
14311,http://zjq.sakura.ne.jp/rawi/the-non-repetitiv...,\n新しい\t新しい\tAi.free\t形容詞-自立\t4\n1\t1\tN.Num\t名...,4
14312,http://www.zushi-kaisei.ac.jp/activity/study/s...,\nまた\tまた\tConj\t接続詞\t4\n、\t、\tSym.c\t記号-読点\t0\...,4
14313,http://zerowaste.jp/cyberaction/nowar/kzm2004/...,\n私\t私\tN.Pron.g\t名詞-代名詞-一般\t4\nも\tも\tP.bind\t...,4
14314,http://zerowaste.jp/cyberaction/nowar/kzm2004/...,\nよく\tよく\tAdv.g\t副詞-一般\t4\nわかり\tわかる\tV.free\t動...,4


In [8]:
# now we make the dataset with only the sentence
df_sentences = df
df_sentences['sentence'] = df['s'].map(extract_sentence)
df_sentences

,url,s,level_old_jlpt,sentence
0,http://www.01-tec.com/document/code_design/vol...,\nよろしく\tよろしく\tAdv.g\t副詞-一般\t4\nお願い\tお願い\tN.Vs\...,4,よろしくお願いします。
1,http://www.0karakasegu.com/,\nいい\tいい\tAi.free\t形容詞-自立\t4\n勉強\t勉強\tN.Vs\t名詞...,4,いい勉強になりました。
2,http://000studio.com/bunka/main/agora/agora02....,\nそれ\tそれ\tN.Pron.g\t名詞-代名詞-一般\t4\nを\tを\tP.c.g\...,4,それを書きましょう。
3,http://www.01-tec.com/book_shop/,\n１\t１\tN.Num\t名詞-数\t0\n冊\t冊\tN.Suff.msr\t名詞-接...,4,１冊目を読んだ人はこちらをどうぞ。
4,http://www.1101.com/saeki/archive/2000-03-22.html,\nなん\tなる\tV.free\t動詞-自立\t4\nな\tだ\tAux\t助動詞\t4\...,4,なんなんだろう。
...,...,...,...,...
14311,http://zjq.sakura.ne.jp/rawi/the-non-repetitiv...,\n新しい\t新しい\tAi.free\t形容詞-自立\t4\n1\t1\tN.Num\t名...,4,新しい1年が今，始まろうとしています。
14312,http://www.zushi-kaisei.ac.jp/activity/study/s...,\nまた\tまた\tConj\t接続詞\t4\n、\t、\tSym.c\t記号-読点\t0\...,4,また、どんな言葉をかけたらいいのかも分からなかった。
14313,http://zerowaste.jp/cyberaction/nowar/kzm2004/...,\n私\t私\tN.Pron.g\t名詞-代名詞-一般\t4\nも\tも\tP.bind\t...,4,私も今となってはわかる。
14314,http://zerowaste.jp/cyberaction/nowar/kzm2004/...,\nよく\tよく\tAdv.g\t副詞-一般\t4\nわかり\tわかる\tV.free\t動...,4,よくわかりますよ。


In [4]:
len(df[df['s'].str.len() > 1]) # big dataset length

TypeError: 'NoneType' object is not subscriptable

In [41]:
# lenghts of the others - maybe could change to dictionary
other_df = [None]
for i in range(1,5):
    print(i)
    with gzip.open(f'./jpWaC-L{i}.vert.gz','rt') as file:
        temp_df = pd.read_xml(file)
        other_df.append(temp_df)
        # print the sentences
        size = len(temp_df[temp_df['s'].str.len() > 1])
        print(f'size of {i}: {size}')

1
size of 1: 31233
2
size of 2: 37365
3
size of 3: 23314
4
size of 4: 14316


In [5]:
df.head(10)

AttributeError: 'NoneType' object has no attribute 'head'

In [23]:
df.loc[0,'s']

'\n藤井\t藤井\tN.Prop.n.s\t名詞-固有名詞-人名-姓\t0\n氏\t氏\tN.Suff.n\t名詞-接尾-人名\t2\nの\tの\tP.prenom\t助詞-連体化\t4\n著書\t著書\tN.g\t名詞-一般\t1\nの\tの\tP.prenom\t助詞-連体化\t4\n販売\t販売\tN.Vs\t名詞-サ変接続\t2\nから\tから\tP.c.g\t助詞-格助詞-一般\t4\n、\t、\tSym.c\t記号-読点\t0\nここ\tここ\tN.Pron.g\t名詞-代名詞-一般\t4\nで\tで\tP.c.g\t助詞-格助詞-一般\t4\nしか\tしか\tP.bind\t助詞-係助詞\t4\n買え\t買える\tV.free\t動詞-自立\t3\nない\tない\tAux\t助動詞\t4\n音声\t音声\tN.g\t名詞-一般\t0\nメール\tメール\tN.Vs\t名詞-サ変接続\t2\nマガジン\tマガジン\tN.g\t名詞-一般\t0\nの\tの\tP.prenom\t助詞-連体化\t4\nコーナー\tコーナー\tN.g\t名詞-一般\t1\nも\tも\tP.bind\t助詞-係助詞\t4\nあり\tある\tV.free\t動詞-自立\t4\nます\tます\tAux\t助動詞\t4\n。\t。\tSym.p\t記号-句点\t0\n'

In [21]:
sentence_test = df.loc[0,'s']

In [25]:
print(sentence_test)


藤井	藤井	N.Prop.n.s	名詞-固有名詞-人名-姓	0
氏	氏	N.Suff.n	名詞-接尾-人名	2
の	の	P.prenom	助詞-連体化	4
著書	著書	N.g	名詞-一般	1
の	の	P.prenom	助詞-連体化	4
販売	販売	N.Vs	名詞-サ変接続	2
から	から	P.c.g	助詞-格助詞-一般	4
、	、	Sym.c	記号-読点	0
ここ	ここ	N.Pron.g	名詞-代名詞-一般	4
で	で	P.c.g	助詞-格助詞-一般	4
しか	しか	P.bind	助詞-係助詞	4
買え	買える	V.free	動詞-自立	3
ない	ない	Aux	助動詞	4
音声	音声	N.g	名詞-一般	0
メール	メール	N.Vs	名詞-サ変接続	2
マガジン	マガジン	N.g	名詞-一般	0
の	の	P.prenom	助詞-連体化	4
コーナー	コーナー	N.g	名詞-一般	1
も	も	P.bind	助詞-係助詞	4
あり	ある	V.free	動詞-自立	4
ます	ます	Aux	助動詞	4
。	。	Sym.p	記号-句点	0



In [28]:
from io import StringIO

# Convert String into StringIO
csvStringIO = StringIO(sentence_test)

df1 = pd.read_csv(csvStringIO, sep='\t', header=None)
df1

,0,1,2,3,4
0,藤井,藤井,N.Prop.n.s,名詞-固有名詞-人名-姓,0
1,氏,氏,N.Suff.n,名詞-接尾-人名,2
2,の,の,P.prenom,助詞-連体化,4
3,著書,著書,N.g,名詞-一般,1
4,の,の,P.prenom,助詞-連体化,4
5,販売,販売,N.Vs,名詞-サ変接続,2
6,から,から,P.c.g,助詞-格助詞-一般,4
7,、,、,Sym.c,記号-読点,0
8,ここ,ここ,N.Pron.g,名詞-代名詞-一般,4
9,で,で,P.c.g,助詞-格助詞-一般,4


In [46]:
# trying with iterparse (more memory efficient)
df_eff = pd.read_xml('./jpWaC-L.vert', iterparse = {"s": ["level"]})

In [47]:
df_eff.shape

(12488610, 1)

In [49]:
df_eff.value_counts()

level
-1       9211681
 0       2627335
 2        372421
 1        137312
 3        103298
 4         36563
Name: count, dtype: int64